In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial= 'http://rev-ib.unam.mx/ib/index.php/ib/issue/archive/2013'
url_root= 'http://rev-ib.unam.mx/ib/index.php/ib/issue/archive/2013'
r=requests.get(url_inicial)

In [ ]:
s = BeautifulSoup(r.text, 'html.parser')

In [ ]:
volumen= s.find('div', id='issues')
v=volumen.findAll('div', id='issue-anterior')
vol = [x.find('a').get('href')for x in v]
vol

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='tocArticle') 
    volumen1=box1.findAll('div', class_='tocTitle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [ ]:
vol2

In [ ]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='tocArticle') 
    volumen1=box1.findAll('div', class_='tocTitle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol2

In [ ]:

links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Investigación Bibliotecológica: Archivonomía, Bibliotecología e Información'
    a='V. Ciencias Sociales'
    te='Ciencias Bibliotecológica y de la Información'
    s_item=BeautifulSoup(r.text,'html.parser')
    
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None 
    #titulo del libro
    titulo=s_item.find('h3').get_text(strip=True)
    if titulo:
        content_book['Titulo Articulo']=titulo
    else :
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', class_='elsevierItemResumenPalabrasResumenTexto').get_text(strip=True)
        content_book['Abstract']=de
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=s_item.find('div', id='content').find('a').get('href')
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    ancla_link=s_item.find('div', id='articleFullText').find('a').get('href')
    if  ancla_link:
        content_book['Link PDF']=ancla_link
    else :
        content_book['Link PDF']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:3]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revista010.9.csv', index=False)